# High-level MXNet MNIST Example

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from keras.datasets import mnist  # Only to import MNIST data

Using CNTK backend
/anaconda/envs/cntk/lib/python3.5/site-packages/cntk/default_options.py:83: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, _, _, _ = getargspec(function_or_class) if isfunction(function_or_class) else getargspec(function_or_class.__init__)
/anaconda/envs/cntk/lib/python3.5/site-packages/keras/legacy/interfaces.py:89: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  wrapper._legacy_support_signature = inspect.getargspec(func)


In [2]:
print(mx.__version__)
print(np.__version__)

0.10.0
1.12.1


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Cast to caffe2 compatible
x_train = np.array(x_train, dtype=np.float32) / 256
x_test = np.array(x_test, dtype=np.float32) / 256
y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)
# Reshape to [N, C, H, W]
x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)
print(x_train.shape)
print(x_test.shape)
print(x_train.dtype)
print(y_train.dtype)

(60000, 1, 28, 28)
(10000, 1, 28, 28)
float32
int32


In [4]:
# Hyperparams
ctx = [mx.gpu(0)]
epochs = 12
batchsize = 32
lr = 0.01
momentum = 0.9
n_classes = 10

In [5]:
def create_lenet():
    # create symbolic representation
    data = mx.symbol.Variable('data')
    input_y = mx.symbol.Variable('softmax_label')  # placeholder for output

    conv1 = mx.symbol.Convolution(
        data=data, kernel=(5,5), num_filter=20)
    tanh1 = mx.symbol.Activation(
        data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(
        data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))

    conv2 = mx.symbol.Convolution(
        data=pool1, kernel=(5,5), num_filter=50)
    tanh2 = mx.symbol.Activation(
        data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(
        data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2)) 

    flatten = mx.symbol.Flatten(
        data=pool2)
    
    fc1 = mx.symbol.FullyConnected(
        data=flatten, num_hidden=500) 
    tanh3 = mx.symbol.Activation(
        data=fc1, act_type="tanh")

    fc2 = mx.symbol.FullyConnected(
        data=tanh3, num_hidden=n_classes) 

    lenet = mx.symbol.SoftmaxOutput(
        data=fc2, label=input_y, name="softmax")
    return lenet

In [6]:
train_iter = mx.io.NDArrayIter(x_train, y_train, batch_size=batchsize, shuffle=True)

In [7]:
model =  mx.mod.Module(context = ctx, symbol = create_lenet())

In [8]:
%%time

# Train
model.fit(
    train_iter,
    optimizer='sgd',
    optimizer_params={'learning_rate':lr,
                      'momentum':momentum},
    eval_metric='acc',
    batch_end_callback = mx.callback.Speedometer(batchsize, 100), 
    num_epoch=epochs
) 

CPU times: user 2min 24s, sys: 18.3 s, total: 2min 42s
Wall time: 2min 20s


In [11]:
# Test model
test_iter = mx.io.NDArrayIter(x_test, y_test, batch_size=batchsize)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
model.score(test_iter, acc)
print("Accuracy ", acc)

Accuracy  EvalMetric: {'accuracy': 0.9906150159744409}
